# Unit 3 part2: Train your Own TensorFlow Image Recognition Model

<img src="img/tensorflow_image_unit3_label1_results1.png">

<img src="img/robotignite_logo_text.png" width="700" />

<b>Estimated time to completion:</b> 2.5-10 hours, depending on the training time

<b>Simulated robot:</b> Mira Robot
<br><br>
<b>What will you learn with this unit?</b>
* Use the trained model in a ROS environment

<p style="background:#AB0017;color:white;">**WARNING**</p><br>

This second part needs the Part1 for it to work, so don't do it until Part1 is ready.

<p style="background:#AB0017;color:white;">**END WARNING**</p><br>

### Step 10: Launch the Testing Training Script

And here comes the **ROS** connection again. So, what we want is for our robot, Mira, to recognise itself in the virtual world. And perhaps in the **real world**. So, we have to make recognitions in real time.

#### Step 10.1: Create the Python script that combines TensorFlow with ROS

This script is a combination of the **test_training.py** and the **image_recognition.py** from **Unit 1**.

<p style="background:#3B8F10;color:white;" id="import_pb_to_tensorboard">**Python Program {3.5-py}: search_for_mira_robot.py** </p>

In [ ]:
#!/usr/bin/env python
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
#from matplotlib import pyplot as plt
#from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import rospkg
import rospy
from sensor_msgs.msg import Image
from std_msgs.msg import String
from cv_bridge import CvBridge
import cv2

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
  
# get an instance of RosPack with the default search paths
rospack = rospkg.RosPack()
# get the file path for rospy_tutorials

path_to_learn_pkg = rospack.get_path('learn_newobjects_tf_pkg')
research_module_path = os.path.join(path_to_learn_pkg,"scripts/models/research")
object_detection_module_path = os.path.join(path_to_learn_pkg,"scripts/models/research/object_detection")
sys.path.append(object_detection_module_path)

#print(sys.path)

from object_detection.utils import ops as utils_ops
from utils import label_map_util

from utils import visualization_utils as vis_util

# What model to download.
MODEL_NAME = 'learned_model'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

scripts_module_path = os.path.join(path_to_learn_pkg,"scripts/")
final_path_to_ckpt = os.path.join(scripts_module_path,PATH_TO_CKPT)

# List of the strings that is used to add correct label for each box. In our case mira_robot
PATH_TO_LABELS = os.path.join(scripts_module_path, 'training/object-detection.pbtxt')

NUM_CLASSES = 1


detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(final_path_to_ckpt, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)
      
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for a single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframing is required to translate the mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict
  

class RosTensorFlow():
    def __init__(self):
        # Processing the variable to process only half of the frame's lower load
        self._process_this_frame = True
        self._cv_bridge = CvBridge()

        self._sub = rospy.Subscriber('image', Image, self.callback, queue_size=1)
        self._pub = rospy.Publisher('result', String, queue_size=1)
        self.score_threshold = rospy.get_param('~score_threshold', 0.1)
        self.use_top_k = rospy.get_param('~use_top_k', 5)
        
        

    def callback(self, image_msg):
        if (self._process_this_frame):
            
            image_np = self._cv_bridge.imgmsg_to_cv2(image_msg, "bgr8")
    
            # Expand dimensions since the model expects images to have shapes: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            output_dict = run_inference_for_single_image(image_np, detection_graph)
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                output_dict['detection_boxes'],
                output_dict['detection_classes'],
                output_dict['detection_scores'],
                category_index,
                instance_masks=output_dict.get('detection_masks'),
                use_normalized_coordinates=True,
                line_thickness=8)
            cv2.imshow("Image window", image_np)
            cv2.waitKey(1)
        else:
            pass
        # We invert it
        self._process_this_frame = not self._process_this_frame
        
        
        
    def main(self):
        rospy.spin()

if __name__ == '__main__':
    rospy.init_node('search_mira_robot_node')
    tensor = RosTensorFlow()
    tensor.main()

<p style="background:#3B8F10;color:white;" id="import_pb_to_tensorboard">**END Python Program {3.5-py}: search_for_mira_robot.py** </p>

#### Step 10.2: Create the launch file for starting the SearchFor MiraRobot

Now, just create the launch file in exactly the same way as you did in Unit 1:

<p style="background:#3B8F10;color:white;" id="start_image_recognition">**Launch File {3.6-launch}: start_search_mira_robot.launch** </p>

<p style="background:#3B8F10;color:white;" id="start_image_recognition">**END Launch File {3.6-launch}: start_search_mira_robot.launch** </p>

And now, you launch it:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
# Step 6: Launch the Testing for Mira in the learn_newobjects_tf_pkg main.launch world
roslaunch tf_unit1_pkg start_search_mira_robot.launch

You now have to go to the **Graphical Tools**:

<img src="img/font-awesome_desktop.png">

<img src="img/tensorflow_image_unit3_label1_results1.png">

<img src="img/tensorflow_image_unit3_label1_results2.png">

### Conclusions of Example:

You probably have seen that teaching this model only **ONE** thing has led to making it an **object** recogniser, rather than  having the ability to differentiate between **mira_robot** and **everything else**.<br>
So, that's the next step that **you will have to do** in the following exercise, **3.2**.

<p style="background:#407EAF;color:white;">END **Example 3.1**</p><br>

<p style="background:#EE9023;color:white;">**Exercise 3.2**</p>

So, now you have to make Mira Robot **differentiate** between **mira_robots** and **other objects**.<br>
This means that you will have to train with **TWO** labels. Therefore, you will have to make all of the necessary modifications so that it trains with the label **mira_robot** and the label **object**.<br>
* To make this task less painful, we have already provided a folder with all of the images labeled with two tags. You can find them in the public git **course_tflow_image_student_data/images_2_labels**.

<p style="background:#EE9023;color:white;">**END Exercise 3.2**</p>

<p style="background:green;color:white;">Solution Exercise 3.2</p>

Please try to do it by yourself, unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png" width="700" />

Follow this link to open the solutions notebook for Unit 3:[solutions_tensofrflow_images_unit3](extra_files/solutions_tensofrflow_images_unit3.ipynb)

The learning process should look something like this:

<img src="img/tensorflow_image_unit3_15hourslearning_labels2.png" width="700" />

This is a learning process of more than 15 hours. But, as you can see, after about the 8th hour, there is no significant improvement.

Your detection process should look something like this:

<img src="img/tensorflow_image_unit3_ex3-2_solution.gif" width="400" />

<p style="background:green;color:white;">END Solution Exercise 3.2</p>